In [1]:
import os

import dill

import matplotlib.pyplot as plt
import torch
import pyro
import copy

import torch
import pyro
import pyro.distributions as dist
import matplotlib.pyplot as plt
import numpy as np

import torch
import time

import pandas as pd
from torch.utils.data import DataLoader

from chirho.indexed.ops import IndexSet, gather
import seaborn as sns


import pyro
from pyro.infer import Predictive

from chirho.counterfactual.handlers import MultiWorldCounterfactual

# from cities.modeling.zoning_models.units_causal_model import UnitsCausalModel
from cities.modeling.zoning_models.distance_causal_model import DistanceCausalModel
from cities.modeling.svi_inference import run_svi_inference
from cities.utils.data_loader import select_from_data


from pyro.infer.autoguide import AutoDiagonalNormal

from cities.modeling.evaluation import (
    prep_data_for_test,
    test_performance,
)

from cities.modeling.zoning_models.tracts_model import TractsModel

from cities.modeling.svi_inference import run_svi_inference
from pyro.infer import Predictive
from chirho.observational.handlers.predictive import PredictiveModel
from chirho.interventional.handlers import do


smoke_test = "CI" in os.environ

# use when testing model health
# smoke_test = True

n_steps = 10 if smoke_test else 1500
num_samples = 10 if smoke_test else 1000

from cities.utils.data_grabber import find_repo_root

root = find_repo_root()

In [2]:

census_tracts_data_path = os.path.join(root, "data/minneapolis/processed/census_tracts_dataset.pt")


ct_dataset_read = torch.load(census_tracts_data_path)

ct_loader = DataLoader(
    ct_dataset_read, batch_size=len(ct_dataset_read), shuffle=True
)

data = next(iter(ct_loader))

print(data["continuous"].keys())
print(data['categorical'].keys())   

kwargs = {
    "categorical": [
        "year",
    ],
    "continuous": {
      'housing_units',
      'total_value',
      'median_value',
      'mean_limit_original',
      'median_distance',
      'income',
      'segregation_original',
      'white_original',
    },
    'outcome': 'housing_units'
}

subset = select_from_data(data, kwargs)
print(subset["continuous"].keys())


tracts_model = TractsModel(
    **subset, categorical_levels=ct_dataset_read.categorical_levels
)

dict_keys(['housing_units', 'housing_units_original', 'income', 'income_original', 'mean_distance', 'mean_distance_original', 'mean_limit', 'mean_limit_original', 'median_distance', 'median_distance_original', 'median_value', 'median_value_original', 'segregation', 'segregation_original', 'total_value', 'total_value_original', 'white', 'white_original'])
dict_keys(['year'])
dict_keys(['housing_units', 'income', 'mean_limit_original', 'median_distance', 'median_value', 'segregation_original', 'total_value', 'white_original'])


In [3]:
pyro.clear_param_store()

guide_path = os.path.join(root, "data/minneapolis/guides", "tracts_model_guide.pkl")

param_path = os.path.join(root, "data/minneapolis/guides", "tracts_model_params.pth")

with open(guide_path, "rb") as file:
    guide = dill.load(file)

pyro.get_param_store().load(param_path)

predictive = Predictive(
    model=tracts_model, guide=guide, num_samples=num_samples, parallel=True
)

In [4]:
values = pd.read_csv(os.path.join(root, "data/minneapolis/processed/values_with_parking.csv"))
display(values.head())

,Unnamed: 0.1,Unnamed: 0,parcel,year,value,avg_value,applied,value_std,avg_value_std,parcel_id,...,distance_to_2015_transit,res_parking_elim_2016_yn,res_parking_reduced_2016_yn,distance_to_2016_transit,distance_to_rail_transit,reform_status,limit,limit_id,limit_con,distance_to_transit
0,0,0,402924140049,2011,99500.0,104000.0,0,-0.103874,-0.106238,0,...,648.530512,False,False,648.530512,6890.397082,0,full,2,1.0,648.530512
1,1,1,1102924340022,2011,69300.0,68840.0,0,-0.118781,-0.123816,1,...,1338.755732,False,True,399.433165,4636.309634,0,full,2,1.0,1338.755732
2,2,2,202824230164,2011,157000.0,179200.0,0,-0.075492,-0.068641,2,...,299.089205,False,True,299.089205,3102.920009,0,full,2,1.0,299.089205
3,3,3,2102824210141,2011,242000.0,271250.0,0,-0.033536,-0.022621,3,...,1566.319470,False,False,1566.319470,7574.651614,0,full,2,1.0,1566.319470
4,4,4,2702924241493,2011,144500.0,129600.0,0,-0.081662,-0.093439,5,...,346.081764,False,True,346.081764,1498.791031,0,eliminated,0,0.0,346.081764


In [5]:
data  = copy.deepcopy(values)

intervention = data['limit_con']

